In [1]:
%load_ext autoreload
%autoreload 2

import os, json
import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl

from ablang_train import ABtokenizer, AbLang, TrainingFrame, CallbackHandler, AbDataModule, ablang_parse_args
from ablang_train import models

/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.9/site-packages/neptune/internal/backends/hosted_client.py:50: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version
/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.9/site-packages/pytorch_lightning/loggers/neptune.py:40: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  from neptune import new as neptune


In [2]:
# SET ARGUMENTS AND HPARAMS
arguments = ablang_parse_args(args=["--json_args", "../reports/model_arguments/00_test_heavy.json"])
arguments.model_specific_args.data_path = '/vols/bitbucket/olsen/processed_oas_data/nov2022/nov2022-paired-all/'#'../data/single_data/'
arguments.model_specific_args.eval_path = '/vols/bitbucket/olsen/processed_oas_data/nov2022/nov2022-paired-all/'
arguments.model_specific_args.use_moe = False
arguments.model_specific_args.mask_technique = 'mix'
arguments.model_specific_args.change_percent = -1
arguments.model_specific_args.over_sample_data = 1
arguments.model_specific_args.variable_masking = True
arguments.model_specific_args

Namespace(name='test', json_args='../reports/model_arguments/00_test_heavy.json', n_encoder_blocks=1, hidden_embed_size=64, n_attn_heads=8, dropout=0.0, use_tkn_dropout=False, loss_fn='CrossEntropy_Loss', a_fn='gelu', fl_gamma=2, use_moe=False, mask_percent=0.15, variable_masking=True, mask_technique='mix', change_percent=-1, leave_percent=0.1, initializer_range=0.02, layer_norm_eps=1e-12, data_path='/vols/bitbucket/olsen/processed_oas_data/nov2022/nov2022-paired-all/', out_path='/data/iraqbabbler/olsen/Documents/projects/AbLang/model-catalogue/paired-ablang/train_ablang_pair/reports/models', eval_path='/vols/bitbucket/olsen/processed_oas_data/nov2022/nov2022-paired-all/', cpus=1, max_fit_batch_size=10, effective_batch_size=20, num_training_steps=1000, warmup_steps=2000, learning_rate=0.0004, cdr3_focus=1, weight_decay=0.01, adam_epsilon=1e-08, adam_betas=[0.9, 0.98], seed=42, eval_batch_size=100, over_sample_data=1, accelerator='cpu', devices=1, precision='32-true', val_check_interval

In [3]:
callbacks = CallbackHandler(
    save_step_frequency=arguments.model_specific_args.log_every_n_steps, 
    progress_refresh_rate=0, 
    outpath=arguments.model_specific_args.out_path
)

In [4]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    # For atomic operations there is currently
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    #
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    np.random.seed(seed)
    
    pl.seed_everything(seed)
    
# SET SEED - IMPORTANT FOR MULTIPLE GPUS, OTHERWISE GOOD FOR REPRODUCIBILITY
enforce_reproducibility(arguments.model_specific_args.seed)

Global seed set to 42


In [5]:
# LOAD AND INITIATE DATA

ablang_dm = AbDataModule(arguments.model_specific_args, ABtokenizer) 
# You are supposed to just be able to add abrep to the fit function, but it doesn't work when using multiple GPUs
ablang_dm.setup('fit')

train = ablang_dm.train_dataloader()
val = ablang_dm.val_dataloader()

In [6]:
for batch in train:
    
    print(batch['input'][0])
    print(batch['labels'][:len(batch['input'][0])])
    
    print((batch['labels'][:len(batch['input'][0])] != -100).sum())
    
    break

tensor([ 0,  6, 15, 10, 20, 15,  6,  7, 12, 12, 12, 20, 15,  4, 13, 12,  2,  7,
         3, 17, 20,  7, 11,  8, 14,  7, 12, 17,  8, 17, 12, 23, 18, 14,  1, 23,
        19, 17,  2, 10, 14, 13, 12, 23, 23, 20,  6, 19,  9, 12, 17, 16,  2,  7,
         4, 23, 18, 12, 12, 14,  8, 23, 18, 14, 14,  7, 15, 23, 12,  2, 17,  8,
        16,  5,  2, 23,  5, 19,  4,  7, 16, 14, 23, 20, 10,  1,  9,  7, 20, 23,
         8,  6,  5, 23, 14, 15, 18, 18, 11,  8, 23, 23, 11, 17, 19,  7,  1, 18,
        12, 13, 18, 19, 18, 12,  1,  5, 15, 19, 12,  4, 12, 23,  8, 15,  8, 15,
        23,  7, 22, 25, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 

In [7]:
arguments.model_specific_args

Namespace(name='test', json_args='../reports/model_arguments/00_test_heavy.json', n_encoder_blocks=1, hidden_embed_size=64, n_attn_heads=8, dropout=0.0, use_tkn_dropout=False, loss_fn='CrossEntropy_Loss', a_fn='gelu', fl_gamma=2, use_moe=False, mask_percent=0.15, variable_masking=True, mask_technique='mix', change_percent=-1, leave_percent=0.1, initializer_range=0.02, layer_norm_eps=1e-12, data_path='/vols/bitbucket/olsen/processed_oas_data/nov2022/nov2022-paired-all/', out_path='/data/iraqbabbler/olsen/Documents/projects/AbLang/model-catalogue/paired-ablang/train_ablang_pair/reports/models', eval_path='/vols/bitbucket/olsen/processed_oas_data/nov2022/nov2022-paired-all/', cpus=1, max_fit_batch_size=10, effective_batch_size=20, num_training_steps=1000, warmup_steps=2000, learning_rate=0.0004, cdr3_focus=1, weight_decay=0.01, adam_epsilon=1e-08, adam_betas=[0.9, 0.98], seed=42, eval_batch_size=100, over_sample_data=1, accelerator='cpu', devices=1, precision='32-true', val_check_interval

In [8]:
# LOAD MODEL
model = TrainingFrame(arguments.model_specific_args, AbLang, ABtokenizer)

# INITIALISE TRAINER
trainer = pl.Trainer(**arguments.trainer_args, callbacks=callbacks())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
%%time
trainer.fit(model, train_dataloaders=train, val_dataloaders=val)


  | Name    | Type             | Params
---------------------------------------------
0 | loss_fn | CrossEntropyLoss | 0     
1 | ablang  | AbLang           | 56.1 K
---------------------------------------------
56.1 K    Trainable params
4         Non-trainable params
56.1 K    Total params
0.224     Total estimated model params size (MB)
/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.9/site-packages/pytorch_lightning/loggers/neptune.py:355: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  self._run_instance = neptune.init_run(**self._neptune_init_args)


https://app.neptune.ai/tobiasheol/AbLangTraining/e/ABLANG-675


/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.9/site-packages/pytorch_lightning/loggers/neptune.py:402: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'list'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  self.run[parameters_key] = params
/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.9/site-packages/pytorch_lightning/loggers/neptune.py:402: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'NoneType'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.

CPU times: user 12min 59s, sys: 1min 8s, total: 14min 7s
Wall time: 1min 48s


In [10]:
seq_to_restore = 'EVQLVESGPGLVQPGKSLRLSCVASGFTFSGYGMHWVRQAPGKGLEWIALIIYDESNKYYADSVKGRFTISRDNSKNTLYLQMSSLRAEDTAVFYCAKVKFYDPTAPNDYWGQGTLVTVSS|'

In [11]:
model1 = model.ablang
tokenizer1 = model.tokenizer

with torch.no_grad():
    tokenPreds = model1(tokenizer1([seq_to_restore], pad=True, w_extra_tkns=False, device='cpu'))

tokenMAX = torch.max(torch.nn.Softmax(dim=-1)(tokenPreds), -1)

aaPreds = tokenizer1(tokenMAX.indices, w_extra_tkns=False, mode='decode', device='cpu')

unkMatrix = torch.zeros(tokenMAX[0].shape, dtype=torch.long, device='cpu') + 21

aaPreds50 = ['-'.join(tokenizer1(torch.where(tokenMAX[0]<=.5, unkMatrix, tokenMAX[1]).detach(), w_extra_tkns=False, mode='decode')[0].split('<unk>'))]

In [12]:
tokenPreds[0][2]

tensor([-1.0831, -0.6939,  0.3854, -1.6172,  0.1254, -0.0360,  0.4362,  1.5897,
         0.7628,  0.2530,  2.0389, -0.5367,  1.6156, -0.1406,  0.5155,  1.2214,
         0.0277, -0.0255,  0.4346,  0.0915,  0.7186, -4.4874, -2.7392, -1.9027,
        -4.6647, -0.5857])

In [13]:
tokenMAX = torch.max(torch.nn.Softmax(dim=-1)(tokenPreds), -1)
tokenMAX[1]

tensor([[ 6, 15, 10, 20, 15,  6,  7, 12,  7, 12, 20, 15,  7,  7, 12,  7,  7, 20,
          2, 20,  7,  7, 15, 14,  7, 12,  7,  8,  7,  7, 12, 18, 12,  7,  7, 19,
         15,  2, 10, 14,  7, 12,  7, 12, 20, 12, 19,  7, 14, 20,  7,  7, 18,  7,
          7,  7,  9,  7, 18, 18, 14,  7,  7, 15,  7, 12,  2,  7,  8,  7,  7,  2,
          7,  7,  7,  7,  7,  8, 20, 18, 20,  7,  7,  7,  7, 20,  2, 14,  7,  7,
          8, 14,  7,  7, 18,  7, 14,  7, 15,  7,  7, 18,  7,  7,  8, 14,  7,  7,
          7, 18, 19, 12, 10, 12,  8, 20, 15,  8, 15,  7,  7, 25]])

In [14]:
masked_to_restore = '***LVESGPGLVQPGKSLRLSCVASGFTFSGYGMHWVRQAPGKGLEWIALIIYDESNKYYADSVKGRFTISRDNSKNTLYLQMSSLRAEDTAVFYCAKVKFYDPTAPND************'
masked_to_restore = '<********************************GMHWVRQAPGKGLEWIALI*********ADSVKGRFTISRDNSKNTLYLQMSSLRA*********************************>|'

In [15]:
with torch.no_grad():
    tokenPreds = model1(tokenizer1([masked_to_restore], pad=True, w_extra_tkns=False, device='cpu'))
    tokenMAX = torch.max(torch.nn.Softmax(dim=-1)(tokenPreds), -1)
    masked_aaPreds = tokenizer1(tokenMAX[1], mode='decode')
masked_aaPreds

['SSSSSSSSSSSSSSSSSSSSSGGGSSGGGGGGGLHAVRQA|GGGLGAIALIGGGGGGSGSADSVGGRLTISRDLSGLTLLLQLSSLRAGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG>']

In [16]:
print(aaPreds[0])
print(aaPreds50[0])
print(seq_to_restore)

EVQLVESGSGLVSSGSSLRLSSVASGSTSSGYGSSWVRQASGSGLGWSALSSYSSSNSYYASSVSGRSTSSRSSSSSTLYLSSSSLRASSTASSYSASVSSYSSTASSSYWGQGTLVTVSS|
-------------------------------------------------------------------------------------------------------------------------|
EVQLVESGPGLVQPGKSLRLSCVASGFTFSGYGMHWVRQAPGKGLEWIALIIYDESNKYYADSVKGRFTISRDNSKNTLYLQMSSLRAEDTAVFYCAKVKFYDPTAPNDYWGQGTLVTVSS|
